In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import nltk 
import string
import re

In [17]:
# Load dataset
def load_data():
    data = pd.read_csv("Digital_Korlantas_reviews_5000_c.csv")
    return data

df = load_data()
df.head()

,content,sentiment
0,Mantap,Positif
1,semoga lebih baik lagi,Positif
2,Ok,Positif
3,Sangat membantu dengan adanya pembuatan sim on...,Positif
4,"Alhamdulillah, SIM sudah mendarat dengan selam...",Positif


In [18]:
 #import stopword
from nltk.corpus import stopwords 
stopwords_indonesia = stopwords.words('indonesian')
 
#import sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

#tokenize
from nltk.tokenize import TweetTokenizer

In [19]:
# Happy Emoticons
emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3'
    ])
 
# Sad Emoticons
emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])
 
# all emoticons (happy + sad)
emoticons = emoticons_happy.union(emoticons_sad)

In [20]:

def clean_content(content):
 
    # remove hyperlinks
    content = re.sub(r'https?:\/\/.*[\r\n]*', '', content)
    
    # remove hashtags
    # only removing the hash # sign from the word
    content = re.sub(r'#', '', content)
    
    #remove coma
    content = re.sub(r',','',content)
    
    #remove angka
    content = re.sub('[0-9]+', '', content)
 
    # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
    tokens = tokenizer.tokenize(content)
    # print("ini token")
    # print(tokens)
 
    content_clean = []    
    for word in tokens:
        if (word not in stopwords_indonesia and # remove stopwords
              word not in emoticons and # remove emoticons
                word not in string.punctuation): # remove punctuation
            #tweets_clean.append(word)
            stem_word = stemmer.stem(word) # stemming word
            content_clean.append(stem_word)
 
    print("ini cleaned")
    print(content_clean)
    return content_clean
df['content_clean'] = df['content'].apply(lambda x: clean_content(x))

ini cleaned
['mantap']
ini cleaned
['moga']
ini cleaned
['ok']
ini cleaned
['bantu', 'buat', 'sim', 'online']
ini cleaned
['alhamdulillah', 'sim', 'darat', 'selamat', 'sampe', 'rumah', 'terima', 'kasih', 'korlantas', 'polri', 'mohon', 'maaf', 'kemaren', 'pesimis', 'aplikasi', 'bantu', 'yg', 'laen', 'mohon', 'sabar', 'daftar', 'mpe', 'sim', 'darat', 'rumah', 'butuh', 'kerja', 'krn', 'lonjak', 'daftar', 'sukses', 'korlantas', 'polri']
ini cleaned
['bagus']
ini cleaned
['foto', 'wajah', 'untk', 'verifikasi', 'nik', 'emang', 'gagal', 'tp', 'pas', 'ganti', 'hp', 'langsung', 'lancar', 'smua', 'proses', 'alhamdulillah', 'on', 'proses', 'bikin', 'cardnya', 'tinggal', 'nunggu', 'd', 'kirim', '', 'makas', 'korlantas', 'polri', 'dah', 'bikin', 'app', 'gk', 'capek', '', 'antre']
ini cleaned
['good']
ini cleaned
['subandi', 'asep', 's kom', 'polri', 'polda']
ini cleaned
['mudah', 'mudah', 'mudah', 'layan', 'masyarakat', 'percaya', 'polri']
ini cleaned
['permisi', 'sim', 'sy', 'habis', 'laku', 'juni

In [21]:
def remove_punct(text):
    text  = " ".join([char for char in text if char not in string.punctuation])
    return text
df['content_clean'] = df['content_clean'].apply(lambda x: remove_punct(x))

In [22]:
sum(df['content_clean']=="")

25

In [23]:
nan_value = float("NaN")
df.replace("", nan_value, inplace=True)
df.dropna(subset = ["content_clean"], inplace=True)
sum(df['content_clean']=="")

0

In [24]:
df.to_csv("digital_korlantas_reviews_5000_p.csv")
# //